## Modelo de Monte Carlo para la valuación de opciones europeas

#### _European Option Pricing with Montecarlo_



#### Ventajas y limitaciones del enfoque de Monte Carlo

#### Ventajas:

- Flexibilidad: el modelo de Monte Carlo es aplicable a una amplia variedad de opciones y activos subyacentes, incluyendo opciones personalizadas que sean más difíciles de adaptar desde Black-Scholes.


#### Limitaciones:

- Tiempo de cálculo: a medida que aumenta el número de simulaciones, el tiempo de cálculo también aumenta.
- Precisión: el valor estimado de la opción puede tener cierta variabilidad debido al carácter aleatorio de las simulaciones.

### Matemáticas necesarias

Puedes valuar una opción sin utilizar el modelo de Black-Scholes y en su lugar para simular el precio del activo subyacente en diferentes momentos, utilizar la ecuación de Movimiento Geométrico Browniano [_(Geometric Brownian Motion)_](https://en.wikipedia.org/wiki/Geometric_Brownian_motion), que se define como:

$$
S_t = S_0 \cdot e^{(r - \frac{\sigma^2}{2})t + \sigma\sqrt{t}Z}
$$


Donde:

$S_t$: Precio del activo subyacente en el tiempo $t$

$S_0$: Precio inicial del activo subyacente

$r$: Tasa de interés libre de riesgo

$\sigma$: Volatilidad del activo subyacente

$t$: Tiempo

$Z$: Variable aleatoria normal estándar (con media 0 y desviación estándar 1)

A continuación, calculamos el valor esperado del pago (valor intrínseco) de la opción en cada trayectoria simulada y descontamos el valor a la fecha presente utilizando la tasa de interés libre de riesgo.

Para una opción europea call, el valor intrínseco es:

$ C_T = max(S_T − K,0) $

Para una opción europea put, el valor intrínseco es:

$ P_T = max(K - S_T,0) $

Donde $K$ es el precio strike de la opción.

El valor esperado de la opción se calcula como la media de los pagos descontados de todas las trayectorias simuladas y se expresa como:

$$
C_0 = e^{-rT} \cdot \frac{1}{N} \sum_{i=1}^{N} C_{T,i}
$$


$$
P_0 = e^{-rT} \cdot \frac{1}{N} \sum_{i=1}^{N} P_{T,i}
$$


**Donde:**

**$C_0$ y $P_0$:** Valor presente de la opción de compra y de venta, respectivamente.

**$T$:** Tiempo hasta el vencimiento de la opción.

**$N$:** Número de trayectorias simuladas.

**$C_{T,i}$ y $P_{T,i}$:** Pago de la opción de compra y de venta en la trayectoria $i$, respectivamente.

$r$: tasa libre de riesgo

##### Pequeña explicación
Estas fórmulas representan el precio actual (también conocido como valor presente) de una cartera de opciones de compra (C) y de venta (P), respectivamente. La estructura de ambas fórmulas es muy similar, y ambas tienen en cuenta el valor presente de los flujos de efectivo futuros.


Para entender mejor cada elemento de las ecuaciones, vamos a desglosarlas:

$C_0$ y $P_0$: estos son los precios actuales de las carteras de opciones de compra y de venta, respectivamente. Se calculan a partir del valor presente de los flujos de efectivo futuros.

$e^{-rT}$: esta es la tasa de descuento, necesaria para convertir a valor presente el valor de las opciones al vencimiento. r es la tasa de interés libre de riesgo, y T es el tiempo en años que falta para el vencimiento de las opciones.

$\frac{1}{N} \sum_{i=1}^{N} C_{T,i}$ y $\frac{1}{N} \sum_{i=1}^{N} P_{T,i}$: estas son las medias aritméticas de los precios de las opciones de compra y de venta que forman la cartera. Cada $C_{T,i}$ y $P_{T,i}$ representa el precio de una opción individual en el momento T.


Ahora, implementemos esto en Python y usemos gráficas para visualizar los resultados.

**Importación de las bibliotecas necesarias**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

**Cálculo del rendimiento diario y la volatilidad anualizada del activo subyacente**

Estas líneas de código se utilizan para calcular la volatilidad anualizada de un activo subyacente, que es un componente clave en la valuación de opciones. Es importante tener en cuenta que el cálculo de la volatilidad implica el uso de logaritmos, que son válidos en el contexto de las finanzas.

El cálculo del rendimiento diario del activo subyacente se realiza a partir de la serie de precios diarios del activo. La función "np.log" se utiliza para calcular el logaritmo natural del cociente de los precios del activo en dos días consecutivos. Al tomar el logaritmo de este cociente, estamos calculando la variación porcentual entre los dos precios y no simplemente la diferencia de precios. Luego, se elimina cualquier valor NaN (not a number) con la función "dropna()" para asegurarnos de que los cálculos posteriores sean precisos.

Una vez que se han calculado los rendimientos diarios del activo subyacente, se utiliza la desviación estándar anualizada de estos rendimientos para calcular la volatilidad anualizada del activo subyacente. La volatilidad anualizada se calcula multiplicando la desviación estándar de los rendimientos diarios por la raíz cuadrada de 252 (que es el número de días de negociación en un año típico). El resultado es la volatilidad anualizada del activo subyacente.

El uso de logaritmos es válido en finanzas porque es común asumir que el rendimiento de los activos financieros tiende a seguir una distribución lognormal. Esto significa que los rendimientos tienden a estar más cerca de la media y tienen una cola más larga hacia los valores extremos. El uso de logaritmos ayuda a manejar los valores extremos de los rendimientos y hacer que los cálculos sean más precisos. Además, el uso de logaritmos permite sumar los rendimientos en lugar de multiplicarlos, lo que hace que los cálculos sean más fáciles.

In [2]:
ticker = 'SPY'
data = yf.download(ticker, start='2022-01-01', end='2025-03-15')['Close']['SPY']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
data

Date
2022-01-03    455.810455
2022-01-04    455.657715
2022-01-05    446.908112
2022-01-06    446.488281
2022-01-07    444.723145
                 ...    
2025-03-10    558.898743
2025-03-11    554.252686
2025-03-12    557.193848
2025-03-13    549.766174
2025-03-14    561.122009
Name: SPY, Length: 802, dtype: float64

In [4]:
daily_returns = data.pct_change()
daily_returns = daily_returns.dropna()
sigma = daily_returns.std() * np.sqrt(252)

In [5]:
S0 = data.iloc[-1] #Precio Spot
r = 0.10 # Tasa libre de riesgo
T = 100 / 365 # Días al vencimiento

num_simulations = 10000


In [6]:
Z = np.random.normal(0, 1, num_simulations)


In [7]:
St = S0 * np.exp( (r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z  )


**Definición de los parámetros para la simulación de Monte Carlo**

**Simulación de Monte Carlo para generar una serie de precios posibles para el activo subyacente en la fecha de vencimiento de las opciones**

Las siguientes dos líneas de código se utilizan para generar trayectorias simuladas del precio del activo subyacente.

La primera línea de código genera un conjunto de números aleatorios normalmente distribuidos utilizando la función "np.random.normal". Estos números se utilizan para simular la variación aleatoria en el precio del activo subyacente. La función "np.random.normal" toma tres argumentos: la media (que es cero en este caso), la desviación estándar (que es uno en este caso) y el número de simulaciones que se desean generar (que es el valor de "num_simulations").

La segunda línea de código utiliza los números aleatorios generados anteriormente para simular los precios del activo subyacente en diferentes momentos en el futuro. La fórmula utilizada se basa en el modelo de movimiento geométrico Browniano.La fórmula tiene en cuenta el precio actual del activo subyacente (S0), la volatilidad anualizada del activo subyacente (sigma), el tiempo hasta el vencimiento de las opciones (T), la tasa de interés libre de riesgo (r) y los números aleatorios generados anteriormente (Z).

**Cálculo del pago de las opciones para cada una de las trayectorias simuladas**

Estas líneas de código se utilizan para calcular el pago de una opción de compra (call) y una opción de venta (put) respectivamente, a partir de los precios simulados del activo subyacente.

La primera línea de código utiliza la función "np.maximum" para calcular el pago de la opción de compra. La función "np.maximum" toma dos argumentos: los precios simulados del activo subyacente (St) y el precio de ejercicio de la opción de compra (K). La función devuelve el valor máximo entre la diferencia entre los precios simulados del activo subyacente y el precio de ejercicio (St - K) y cero. Esto asegura que el pago de la opción de compra siempre sea cero si el precio del activo subyacente en el futuro es menor que el precio de ejercicio, y de lo contrario, sea igual a la diferencia entre el precio simulado del activo subyacente y el precio de ejercicio.

La segunda línea de código utiliza la misma función "np.maximum" para calcular el pago de la opción de venta. En este caso, la función devuelve el valor máximo entre la diferencia entre el precio de ejercicio y los precios simulados del activo subyacente (K - St) y cero. Esto asegura que el pago de la opción de venta siempre sea cero si el precio del activo subyacente en el futuro es mayor que el precio de ejercicio, y de lo contrario, sea igual a la diferencia entre el precio de ejercicio y el precio simulado del activo subyacente.

In [8]:
K = S0

In [9]:
S0

561.1220092773438

In [10]:
call = np.maximum(St - K, 0)
put = np.maximum(K - St, 0)

In [11]:
call

array([45.46670831, 12.74193684, 50.66788608, ..., 66.0380256 ,
        0.        ,  0.71885736])

In [12]:
put

array([0.        , 0.        , 0.        , ..., 0.        , 3.58166556,
       0.        ])

In [13]:
call_value = np.exp(-r * T) * np.mean(call)
put_value = np.exp(-r * T) * np.mean(put)

In [14]:
# K = 580
call_value

28.267274757252014

In [15]:
# K = 580
put_value

13.488079163082753

**Cálculo del valor presente de las opciones mediante la obtención del promedio de los pagos de las opciones y su descontado a su valor presente**

Estas líneas de código se utilizan para calcular el valor presente de una opción de compra (call) y una opción de venta (put), respectivamente, a partir de los pagos de opciones calculados previamente y teniendo en cuenta la tasa de interés libre de riesgo.

La primera línea de código utiliza la fórmula de valor presente para calcular el valor presente de la opción de compra. La fórmula tiene en cuenta el pago de la opción de compra calculado previamente (call_payoff), la tasa de interés libre de riesgo (r) y el tiempo hasta el vencimiento de la opción (T). La función "np.mean" se utiliza para calcular la media aritmética de los pagos de opciones. Luego, se multiplica el valor promedio de los pagos de opciones por el factor de descuento del valor presente (np.exp(-r * T)) para obtener el valor presente de la opción de compra.

La segunda línea de código utiliza la misma fórmula de valor presente para calcular el valor presente de la opción de venta. En este caso, se utiliza el pago de la opción de venta calculado previamente (put_payoff) en lugar del pago de la opción de compra. Se utiliza la misma función "np.mean" para calcular la media aritmética de los pagos de opciones, y luego se multiplica por el factor de descuento del valor presente (np.exp(-r * T)) para obtener el valor presente de la opción de venta.

**Impresión de los resultados de la valuación de las opciones**